# PHYS 1600 HW1
Tim Zhao, B01307256, Feb 10, 2021

In [49]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

## #1 
The Debye model of a solid is used to estimate the contribution of atomic vibrations (phonons) to the specific heat capacity. In the Debye model, the specific heat capacity of a solid is given by the following integral expression: 
$C_v=9k_B\bigg(\frac{T}{\theta_D}\bigg)\int_{0}^{\theta_D/T} \frac{x^4e^x}{(e^x-1)^2}dx$ 
where kB is Boltzmann’s constant and θD is the Debye temperature, a property of solids that depends on their density and atomic bonding strength.

a) Write a Python function to calculate $C_V$ for a given material and temperature. Your program should take the density, sample volume, Debye Temperature, temperature upper limit, and required accuracy as inputs, and output the heat capacity between $T = 0$ and the upper temperature limit. Use an adaptive Simpson’s rule to evaluate the integral to the required accuracy.

In [50]:
def adapt_simp(f, start, stop, accuracy, args=None):
    """
    f - function to integrate
    start - lower limit
    stop - upper limit
    accuracy - desired error range
    """
    
    #set initial h and error
    h = 0.1
    error = 100
    
    #find S0 w/ simpsons
    x_list = np.arange(start,stop+h,h)
    S = (1/3)*(f(start,*args)+f(stop,*args)+2*f(x_list[2:-2:2],*args).sum())
    T = (2/3)* f(x_list[1:-1:2],*args).sum()
    F_current = h*(S+2*T)

    while error > accuracy:
        F_last = F_current
        #half step size
        h /= 2
        x_list = np.arange(start,stop+h,h)
        
        #adaptive simpson
        S += T
        T = (2/3)* f(x_list[1:-1:2],*args).sum()

        F_current = h*(S+2*T)
        
        #check error
        error = abs((1/15)*(F_current-F_last))
    
    return (F_last, error)

In [51]:
"""
#test adaptive simpson
def Gaussian(x,amp,mean,std_dev):
    f = amp*np.exp(-1*((x - mean)**2/(2*std_dev**2))) 
    return f

simps = adapt_simp(Gaussian, -10, 10, 0.0001, args = [1,0,1])
print(simps)

scipyIntegral = integrate.quad(Gaussian, -100.0, 100.0, args = (1,0,1))
print(scipyIntegral)
"""

'\n#test adaptive simpson\ndef Gaussian(x,amp,mean,std_dev):\n    f = amp*np.exp(-1*((x - mean)**2/(2*std_dev**2))) \n    return f\n\nsimps = adapt_simp(Gaussian, -10, 10, 0.0001, args = [1,0,1])\nprint(simps)\n\nscipyIntegral = integrate.quad(Gaussian, -100.0, 100.0, args = (1,0,1))\nprint(scipyIntegral)\n'

In [54]:
k_B=1.38 * 10**(-23)
Avogadro = 6.02 * 10**23.
def integrant(x,a,b):
    g=(x**a*np.exp(x))/(np.exp(x)-1)**b
    return g

def Debye(rho,V,molar_mass,theta_D,T_max,accuracy):
    """    
    rho - density
    V - volume
    theta_D - Debye Temperature
    T_max - Temperature upper limit
    accu - required accuracy
    return array of Temperature and Heat Capacity between T=0 and T_max
    """    
    #initialize empty lists to store data
    Cv_list=[]
     
    for T in T_list:
        Cv=(rho*V)*9*k_B*(T/theta_D)**3*adapt_simp(integrant, 0.0001,(theta_D/T), accuracy,[4,2])[0]
        Cv_list.append(Cv)
    
    return Cv_list

In [55]:
density=10**-6
T_max=200
T_list=np.arange(0,T_max,0.01)
accuracy=0.01

Pb_rho=11343
Pb_molar_mass=207.2
Pb_theta_D=105
Pb_Debye=Debye(Pb_rho, density, Pb_molar_mass,Pb_theta_D, T_max, accuracy)
plt.plot(T_list,Pb_Debye,label='Pb')

#Al_Debye=Debye(2.7,1,428,2000,0.0000001)
#diamond_Debye=Debye(3.51,1,2230,2000,0.0000001)
#print(Pb_Debye)

#plt.plot(Al_Debye,label='Al')
#plt.plot(Diamond_Debye,label='Diamond')
plt.show()
print('1')

<ipython-input-54-562e799a0de4>:20: RuntimeWarning: divide by zero encountered in double_scalars
  Cv=(rho*V)*9*k_B*(T/theta_D)**3*adapt_simp(integrant, 0.0001,(theta_D/T), accuracy,[4,2])[0]


ValueError: Maximum allowed size exceeded

In [66]:
#General ODE solver with Runge-Kutta 4
def RK4(f,t0,t_f,dt,x0,arg=None):
    """
    f - a function, takes 2 variables (t,x), RHS of ODE
    t0 - lower limit of independent variable
    t_f - upper limit of independent variable
    dt - step size
    x0 - initial value 
    return - array of t and x
    """
    
    # number of time steps, n must be an integer
    n  = int(np.ceil((t_f-t0)/dt))
    
    # create empty lists to store values
    x_list=np.zeros(n, np.longdouble)
    t_list=np.zeros(x_list.size)
    
    # set initial conditions at time zero
    x_list[0] = x0
    t_list[0] = t0
    
    # RK4
    for ii in range(n-1):
            k1 = dt*f(x_list[ii],t_list[ii],*args)
            k2 = dt*f(x_list[ii]+0.5*k1,t_list[ii]+dt/2,*args)
            k3 = dt*f(x_list[ii]+0.5*k2,t_list[ii]+dt/2,*args)
            k4 = dt*f(x_list[ii]+0.5*k3,t_list[ii]+dt/2,*args)
            
            # update value of Q at each time step and step time by dt
            x_list[ii+1]=x_list[ii]+(k1+2*k2+2*k3+k4)/6
            t_list[ii+1]=t_list[ii]+dt
            
    return t_list, x_list

# #2
A simple electronic circuit with a resistor and capacitor is shown below.
A voltage is sent into the circuit on the left at $V_{in}$ and read out on the right at $V_{out}$.

a)Find the 1st order ordinary differential equation governing $V_{out}(t)$. Assume the output load impedance is infinite so that no current flows through it. Your equation should relate $dV_{out}/dt$ to $R$, $C$, and $V_{in} − V_{out}$.



$\frac{dQ}{dt}=I=\frac{V_{in}-V_{out}}{R}$

$V_{out}=\frac{Q}{C}$

$\frac{dQ}{dt}=\frac{d}{dt}V_{out}C$

$\frac{d}{dt}V_{out}=\frac{V_{in}-V_{out}}{RC}$

b) Use your RK4 integrator to solve the equation from (a) for a square wave input signal, with frequency 1 and amplitude 1. Make plots of Vout(t) from t = 0 to t = 10 when RC = 0.01, 0.1, and 1, with initial condition Vout(0) = 0. Be careful to select an appropriate time step size for your solution in each case.

In [72]:
#make sqaure wave
from scipy import signal

def circuit(V_out, t, RC):
    V_in = signal.square(2 * np.pi * t)
    return (V_in-V_out)/RC

RC1 = RK4(circuit, 0, 10, 0.001,[np.arange(0,10,0.001),1])
#RC01 = RK4(circuit, 0.1,  0, 1, 0)
#RC001 = RK4(circuit, 0.01, 0, 10, 0)

TypeError: float() argument must be a string or a number, not 'list'